In [ ]:
!pip install opencv-python
pip install opencv-python mediapipe


In [1]:
import cv2

# Open default webcam (0 = first camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Webcam not accessible")
else:
    print("✅ Webcam is working. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to grab frame")
            break

        # Display the frame
        cv2.imshow("Webcam Test", frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release camera and close window
cap.release()
cv2.destroyAllWindows()


✅ Webcam is working. Press 'q' to quit.


In [2]:
import cv2
import mediapipe as mp
import time

# Setup webcam
cap = cv2.VideoCapture(0)

# Mediapipe modules
mp_face = mp.solutions.face_detection
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

face_detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)
hands_detector = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Toggle flags
show_face = False
show_hands = False
img_counter = 0

print("✅ Press 's' = Save image | 'f' = Toggle Face | 'h' = Toggle Hands | 'q' = Quit")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to capture frame")
        break

    # Flip for selfie view
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Face detection
    if show_face:
        face_results = face_detector.process(rgb)
        if face_results.detections:
            for detection in face_results.detections:
                mp_draw.draw_detection(frame, detection)

    # Hand joint detection
    if show_hands:
        hand_results = hands_detector.process(rgb)
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Show frame
    cv2.imshow("Webcam Feed", frame)

    # Key controls
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('s'):
        filename = f"snapshot_{img_counter}.png"
        cv2.imwrite(filename, frame)
        print(f"📸 Saved: {filename}")
        img_counter += 1
    elif key == ord('f'):
        show_face = not show_face
        print(f"🧍 Face Detection: {'ON' if show_face else 'OFF'}")
    elif key == ord('h'):
        show_hands = not show_hands
        print(f"🖐️ Hand Detection: {'ON' if show_hands else 'OFF'}")

# Clean up
cap.release()
cv2.destroyAllWindows()


✅ Press 's' = Save image | 'f' = Toggle Face | 'h' = Toggle Hands | 'q' = Quit
🧍 Face Detection: ON
🧍 Face Detection: OFF
🧍 Face Detection: ON
🧍 Face Detection: OFF
🖐️ Hand Detection: ON
🖐️ Hand Detection: OFF
🧍 Face Detection: ON
🧍 Face Detection: OFF
🧍 Face Detection: ON
🧍 Face Detection: OFF


In [5]:
import cv2
import mediapipe as mp
import time
import datetime

# Setup
cap = cv2.VideoCapture(0)
frame_width, frame_height = int(cap.get(3)), int(cap.get(4))

# Flags
show_face = False
show_hands = False
show_pose = False
show_mesh = False
show_time = False
recording = False
img_counter = 0
video_writer = None

# Mediapipe modules
mp_draw = mp.solutions.drawing_utils
mp_face = mp.solutions.face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
mp_hands = mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_pose = mp.solutions.pose.Pose()
mp_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

print("""
✅ Webcam Ready
Press:
  q = Quit
  s = Save Image
  v = Start/Stop Video Recording
  f = Toggle Face Detection
  h = Toggle Hand Detection
  p = Toggle Pose Detection
  e = Toggle Face Mesh
  t = Toggle Timestamp
""")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Face detection
    if show_face:
        results = mp_face.process(rgb)
        if results.detections:
            for detection in results.detections:
                mp_draw.draw_detection(frame, detection)

    # Hand detection
    if show_hands:
        results = mp_hands.process(rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)

    # Body pose detection
    if show_pose:
        results = mp_pose.process(rgb)
        if results.pose_landmarks:
            mp_draw.draw_landmarks(frame, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # Face mesh
    if show_mesh:
        results = mp_mesh.process(rgb)
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_draw.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_draw.DrawingSpec(thickness=1, circle_radius=1)
                )

    # Timestamp overlay
    if show_time:
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cv2.putText(frame, timestamp, (10, frame_height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show frame
    cv2.imshow("Live Feed", frame)

    # Write to video file
    if recording and video_writer is not None:
        video_writer.write(frame)

    key = cv2.waitKey(1) & 0xFF

    # Keyboard controls
    if key == ord('q'):
        break
    elif key == ord('s'):
        filename = f"snapshot_{img_counter}.png"
        cv2.imwrite(filename, frame)
        print(f"📸 Image saved: {filename}")
        img_counter += 1
    elif key == ord('v'):
        if not recording:
            out_file = f"recording_{int(time.time())}.avi"
            video_writer = cv2.VideoWriter(out_file, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (frame_width, frame_height))
            recording = True
            print(f"🔴 Started recording: {out_file}")
        else:
            recording = False
            video_writer.release()
            video_writer = None
            print("⏹️ Stopped recording")
    elif key == ord('f'):
        show_face = not show_face
        print(f"🧍 Face Detection: {'ON' if show_face else 'OFF'}")
    elif key == ord('h'):
        show_hands = not show_hands
        print(f"🖐️ Hand Detection: {'ON' if show_hands else 'OFF'}")
    elif key == ord('p'):
        show_pose = not show_pose
        print(f"🏃 Body Pose: {'ON' if show_pose else 'OFF'}")
    elif key == ord('e'):
        show_mesh = not show_mesh
        print(f"🧠 Face Mesh: {'ON' if show_mesh else 'OFF'}")
    elif key == ord('t'):
        show_time = not show_time
        print(f"⏰ Timestamp: {'ON' if show_time else 'OFF'}")

# Cleanup
cap.release()
if video_writer:
    video_writer.release()
cv2.destroyAllWindows()



✅ Webcam Ready
Press:
  q = Quit
  s = Save Image
  v = Start/Stop Video Recording
  f = Toggle Face Detection
  h = Toggle Hand Detection
  p = Toggle Pose Detection
  e = Toggle Face Mesh
  t = Toggle Timestamp

🖐️ Hand Detection: ON
⏰ Timestamp: ON
🏃 Body Pose: ON
🧍 Face Detection: ON
🧍 Face Detection: OFF
⏰ Timestamp: OFF
🏃 Body Pose: OFF
🧍 Face Detection: ON
🧍 Face Detection: OFF
🧍 Face Detection: ON
🧍 Face Detection: OFF
🧍 Face Detection: ON
🧍 Face Detection: OFF
